In [ ]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import os
import sys
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from darts.timeseries import TimeSeries
from darts.utils.statistics import (
    check_seasonality,
    stationarity_tests,
    plot_acf,
    plot_pacf,
)
import statsmodels.api as sm
import numpy as np

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.vars import DATASET_FILES

plt.style.use("default")
DATA_DIR = Path("../data")

In [ ]:
def plot_seasonal_decompose(result, title="Seasonal Decomposition"):
    return (
        make_subplots(
            rows=4,
            cols=1,
            subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
        )
        .add_trace(
            go.Scatter(x=result.seasonal.index, y=result.observed, mode="lines"),
            row=1,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.trend.index, y=result.trend, mode="lines"),
            row=2,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.seasonal.index, y=result.seasonal, mode="lines"),
            row=3,
            col=1,
        )
        .add_trace(
            go.Scatter(x=result.resid.index, y=result.resid, mode="lines"),
            row=4,
            col=1,
        )
        .update_layout(
            height=900, title=title, margin=dict(t=100), title_x=0.5, showlegend=False
        )
    )

In [ ]:
for dataset_file in DATASET_FILES:
    dataset_file = Path(dataset_file)
    # read csv
    df = pd.read_csv(DATA_DIR / dataset_file)
    print(f"File: {dataset_file.stem}")

    # target col
    target_col = df.select_dtypes(include="number").columns
    assert len(target_col) == 1, f"Expected 1 number column, got {len(target_col)}"
    target_col = target_col[0]

    # any missing values
    print(f"Number of missing values in {target_col}: {df[target_col].isnull().sum()}")

    # date col
    time_col = df.columns.tolist()
    time_col.remove(target_col)
    assert len(time_col) == 1, f"Expected 1 datetime column, got {len(time_col)}"
    time_col = time_col[0]
    df[time_col] = pd.to_datetime(df[time_col])

    # any missing dates
    print(
        f"Missing date in {time_col}: {pd.date_range(start = df[time_col].min(), end = df[time_col].max(), freq=pd.infer_freq(df[time_col])).difference(df[time_col])}"
    )

    # sort
    df = df.sort_values(time_col).reset_index(drop=True)

    # prepare cols for later use
    df["weekends"] = df[time_col].apply(lambda x: 0 if x.weekday() < 5 else 1)

    # fill missing values
    df[target_col] = df[target_col].ffill()

    # stats after filling missing values
    display(df.describe().round(2))

    # after setting index, create darts time series for darts
    df.set_index([time_col], inplace=True)
    df_darts = TimeSeries.from_dataframe(df, value_cols=target_col)

    # statistics
    print(f"Time column: min: {df.index.min()}; max:{df.index.max()}")

    # check moving average
    print(
        f"The moving average of : {df[target_col].rolling(window=3).mean().iloc[:5].round(2)}"
    )

    print(
        f"Forecast the next 1 data point using the naïve method. {df[target_col].tail(1).round(2)}"
    )
    print(
        f"Forecast the next 1 data point using the average method. {df[target_col].mean().round(2)}"
    )

    # check seasonality
    is_seasonality, seasonal_period = check_seasonality(df_darts, max_lag=len(df))
    print(
        f"Is there any seasonality: {is_seasonality}; Seasonal period: {seasonal_period}"
    )
    print(
        f"If the time series is stationary by adf and kpss: {stationarity_tests(df_darts)}"
    )

    # check correlation for weekends
    print(
        f'Linear correlation between weekends: {df[target_col].corr(df["weekends"], method="pearson").round(2)}'
    )

    # statistics
    print(f"The first 10 data points: {df.head(10).describe().round(2)}")
    print(
        "What is the total of the maximum of the first 10 data points and the minimum of the last 10 data points?"
    )
    print(f"{(df[target_col].head(10).max() + df[target_col].tail(10).min()).round(2)}")

    print(
        f'total amount when the day is Tuesday: {df[df.index.day_name() == 'Tuesday'][target_col].sum().round(2)}'
    )

    # calc IQR
    Q1 = df[target_col].quantile(0.25)
    Q3 = df[target_col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print(
        "How many outliers?",
        df[(df[target_col] < lower) | (df[target_col] > upper)].shape[0],
    )

    # get cross correlation
    acf, _ = sm.tsa.acf(df[target_col], alpha=0.05, nlags=10)
    acf = np.absolute(acf)
    print(
        f"Absolute autocorrelation with the weakest lagging: {np.where(acf == min(acf))[0][0]}"
    )

    # naive forecasting for the next data point
    print(f"Naive forecasting: {df.iloc[-1][target_col].round(2)}")

    #  forecasting for the next data point
    print(f"Average forecasting: {df[target_col].mean().round(2)}")
    print(
        f"Average exclude the last data point: {df[target_col].iloc[:-1].mean().round(2)}"
    )

    # plot seasonal decomposition
    # for mode in ["additive", "multiplicative"]:
    #     tmp = df.copy()
    #     if mode == "multiplicative" and df[target_col].min() <= 0:
    #         tmp[target_col] = tmp[target_col] + tmp[target_col].abs().min() + 1
    #     decomposition = seasonal_decompose(
    #         tmp[target_col], model=mode, period=seasonal_period
    #     )
    #     plot_seasonal_decompose(
    #         result=decomposition, title=f"{mode}_{Path(dataset_file).stem}"
    #     ).show()

    # plot acf and pacf for 2 cycles of seasonality
    plot_acf(df_darts, max_lag=seasonal_period * 2)
    plot_pacf(df_darts, max_lag=seasonal_period * 2)

    print("=" * 50)